In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install langchain
!pip install torch
!pip install accelerate
!pip install sentence-transformers
!pip install streamlit
!pip install streamlit-chat
!pip install faiss-cpu
!pip install tiktoken
!pip install huggingface-hub
!pip install pypdf
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  U

In [3]:
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
import os
import tempfile

In [9]:
def initialize_session_state():
    session_state = {
        "history": [],
        "generated": ["Hello! Ask me anything about 🤖"],
        "past": ["Hey! 👋"]
    }
    return session_state


In [10]:
def conversation_chat(query, chain, session_state):
    result = chain({"question": query, "chat_history": session_state["history"]})
    session_state["history"].append((query, result["answer"]))
    return result["answer"]


In [11]:
def display_chat_history(chain, session_state):
    user_input = input("Enter your question: ")
    if not user_input:
        return

    print(f"Repeating question '{user_input}' 10 times:")

    for _ in range(10):
        output = conversation_chat(user_input, chain, session_state)
        print("Bot:", output)



In [12]:
def create_conversational_chain(vector_store):
    print('Creating conversational chain...')
    print("Started creating LLM...")

    llm = LlamaCpp(
        streaming=True,
        model_path="/content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
        temperature=0.75,
        top_p=1,
        verbose=True,
        n_ctx=4096,
    )

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
        memory=memory,
    )

    print("Completed creating LLM!")

    return chain

In [ ]:
def main():
    session_state=initialize_session_state()
    print("ChatBot using Mistral-7B-Instruct LLM :books:")

    uploaded_files = ["/content/University2.pdf"]

    if uploaded_files:
        text = []
        for file_path in uploaded_files:
            with open(file_path, "rb") as f:
                file_contents = f.read()
            file_extension = os.path.splitext(file_path)[1]
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(file_contents)
                temp_file_path = temp_file.name
                print("loading: ", file_path)
            loader = None
            if file_extension == ".pdf":
                loader = PyPDFLoader(temp_file_path)

            if loader:
                text.extend(loader.load())
                os.remove(temp_file_path)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=10000, chunk_overlap=20
        )
        text_chunks = text_splitter.split_documents(text)

        print("chunks:\n", text_chunks)

        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": "cpu"},
        )

        print("embeddings:\n", embeddings)

        vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

        chain = create_conversational_chain(vector_store)

        display_chat_history(chain,session_state)


if __name__ == "__main__":
    main()

ChatBot using Mistral-7B-Instruct LLM :books:
loading:  /content/University2.pdf
chunks:
 [Document(page_content='The Faculty of Information Technology is the youngest of the three faculties of the  University \nof Moratuwa  and is the first ever faculty of its kind in the national university system of Sri \nLanka. The Faculty was established in June 2001 to cater to the growing need of Information \nTechnology professionals in the country. Presently, the Faculty conducts two int ernal degree \nprogrammes, namely,  B.Sc. (Hons.) in Information Technology  and B.Sc. (Hons.) in Information \nTechnology & Management , and the  Bachelor of Information Technology , the external degree \nprogramme.  \n \nOne of the main objectives of setting up the faculty is to increase the number of IT \nprofessionals in Sri Lanka by at least another 500. Some of the other objectives defined during \nthe establishment of the faculty are the introduction of postgraduate progr ammes, carrying \nout research 

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:    

Completed creating LLM!
Enter your question: What are the three departments in the IT faculty?
Repeating question 'What are the three departments in the IT faculty?' 10 times:
